In [119]:
# Import packages and modules
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [120]:
df  = pd.read_csv("../Data/dataset_dk3619_preprocessed_v2_dropnan.csv")

In [121]:
df.shape

(3322211, 43)

In [122]:
print(df['DK3619Code'])
dkcode_list = list(set(df['DK3619Code'].to_list()))
dkcode_list.sort()
id2dkcode = enumerate(dkcode_list)
id2dkcode = dict(id2dkcode)

dkcode2id = {}
for i in id2dkcode.items():
    dkcode2id[i[1]] = i[0]


print(dict(dkcode2id))

### Mapping data to numberic
# df['DK3619Code'] = df['DK3619Code'].map(dkcode2id)

0                 A_A
1                C_CA
2                C_CC
3          C_CE_CF_CD
4                C_CG
              ...    
3322206          Q_QB
3322207           R_R
3322208           S_S
3322209    C_CE_CF_CD
3322210          M_MA
Name: DK3619Code, Length: 3322211, dtype: object
{'A_A': 0, 'C_CA': 1, 'C_CC': 2, 'C_CE_CF_CD': 3, 'C_CG': 4, 'C_CH': 5, 'C_CI': 6, 'C_CJ': 7, 'C_CK': 8, 'C_CL': 9, 'C_CM_CB': 10, 'D_D': 11, 'E_E': 12, 'F_F': 13, 'G_G': 14, 'H_H': 15, 'I_I': 16, 'J_JA': 17, 'J_JB_JC': 18, 'K_K': 19, 'L_L': 20, 'M_MA': 21, 'M_MB': 22, 'M_MC': 23, 'N_N': 24, 'O_O': 25, 'PR_PR': 26, 'P_P': 27, 'Q_QA': 28, 'Q_QB': 29, 'R_R': 30, 'S_S': 31}


In [123]:
### Season mapping 

seasons_mapping = {
    'Spring': 0,
    'Summer': 1,
    'Autumn': 2,
    'Winter': 3
}

df['season'] = df['season'].map(seasons_mapping)

In [124]:
# Export the dkcode_list to a CSV file
dkcode_df = pd.DataFrame(dkcode_list, columns=['DK3619Code'])
dkcode_df.to_csv('dkcode_list.csv', index=False)

### Choose Feauture will use for Model

In [125]:
columns = ['HourUTC', 'HourDK', 'hour', 'day_of_week', 'month', 'year', 'day_of_year', 'day_of_month', 'season', 'holiday', 'lag_24', 'Consumption_MWh']


In [126]:
df_corr = df[columns]
df_corr.head()

,HourUTC,HourDK,hour,day_of_week,month,year,day_of_year,day_of_month,season,holiday,lag_24,Consumption_MWh
0,2024-06-19T21:00:00,2024-06-19 23:00:00,23,2,6,2024,171,19,1,0,147.836362,144.276350
1,2024-06-19T21:00:00,2024-06-19 23:00:00,23,2,6,2024,171,19,1,0,227.710918,231.498108
2,2024-06-19T21:00:00,2024-06-19 23:00:00,23,2,6,2024,171,19,1,0,44.204254,48.562569
3,2024-06-19T21:00:00,2024-06-19 23:00:00,23,2,6,2024,171,19,1,0,166.322232,169.283123
4,2024-06-19T21:00:00,2024-06-19 23:00:00,23,2,6,2024,171,19,1,0,131.194223,131.341630


In [127]:
# Convert holiday to binary indicator
df_corr['holiday'] = df_corr['holiday'].apply(lambda x: 0 if x == 0 else 1)

/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/3026962626.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['holiday'] = df_corr['holiday'].apply(lambda x: 0 if x == 0 else 1)


### Split data

In [128]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU, Dense, Bidirectional
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.layers import TimeDistributed, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

import os.path


In [129]:
# Create sequences
def create_sequences(df, seq_length):
    sequences = []
    labels = []
    for i in range(len(df) - seq_length):
        seq = df.iloc[i:i+seq_length].values
        label = df['Consumption_MWh'].iloc[i+seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [130]:
#all_dkcode_trainings
if not os.path.exists("../Checkpoints/"):
    os.makedirs("../Checkpoints/")

for i in dkcode_list:
    if os.path.exists("../Checkpoints/" + i + "_LSTMmodel.h5"):
        continue
    print('Training LSTM model with DKCode: ', i)
    df_filter = df[df['DK3619Code'] == i]
    df_corr = df_filter[columns]
    df_corr['lag_24'] = df_corr['lag_24'].fillna(0)

    df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
    df_corr['HourDK'] = pd.to_datetime(df_corr['HourDK'])

    df_corr = df_corr.drop(['HourUTC', 'HourDK'], axis=1)

    # Normalize the data
    scaler = MinMaxScaler()
    
    scaled_df = scaler.fit_transform(df_corr[['hour', 'day_of_week', 'month', 'year', 'day_of_year', 'day_of_month', 'season', 'holiday', 'lag_24', 'Consumption_MWh']])

    scaled_df = pd.DataFrame(scaled_df, columns=['hour', 'day_of_week', 'month', 'year', 'day_of_year', 'day_of_month', 'season', 'holiday', 'lag_24', 'Consumption_MWh'])

    #scaled_df = scaler.fit_transform(df_corr[['hour', 'day_of_week', 'month', 'year', 'day_of_year', 'day_of_month', 'season', 'holiday', 'Consumption_MWh']])

    #scaled_df = pd.DataFrame(scaled_df, columns=['hour', 'day_of_week', 'month', 'year', 'day_of_year', 'day_of_month', 'season', 'holiday', 'Consumption_MWh'])
    
    #create sequence for training
    SEQ_LENGTH = 24  # for 24 hours sequence
    sequences, labels = create_sequences(scaled_df, SEQ_LENGTH)

    # Split the data into training (60%), validation (20%), and test (20%) sets
    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, shuffle=False)
    print(X_train.shape[2])
    #LSTM modeling
    #SEQ_LENGTH=24 and  X_train.shape[2] = 6
    model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, X_train.shape[2])),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

    # Model Training
    history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

    model.save("../Checkpoints/" + i + "_LSTMmodel.h5")
    model = tf.keras.models.load_model("../Checkpoints/" + i + "_LSTMmodel.h5")


    # Model Evaluation
    predictions = model.predict(X_test)

    # Calculate evaluation metrics
    r2 = r2_score(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)

    print(f'R² Score: {r2}')
    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'Mean Squared Error (MSE): {mse}')
    print(f'Root Mean Squared Error (RMSE): {rmse}')

Training LSTM model with DKCode:  C_CJ


/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/1540299301.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['lag_24'] = df_corr['lag_24'].fillna(0)
/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/1540299301.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/1540299301.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

10
Epoch 1/10
4224/4224 [==============================] - 29s 7ms/step - loss: 0.0046 - val_loss: 0.0011 - lr: 0.0010
Epoch 2/10
4224/4224 [==============================] - 26s 6ms/step - loss: 0.0011 - val_loss: 6.0323e-04 - lr: 0.0010
Epoch 3/10
4224/4224 [==============================] - 26s 6ms/step - loss: 7.6846e-04 - val_loss: 3.3505e-04 - lr: 0.0010
Epoch 4/10
4224/4224 [==============================] - 26s 6ms/step - loss: 6.4410e-04 - val_loss: 2.1109e-04 - lr: 0.0010
Epoch 5/10
4224/4224 [==============================] - 27s 6ms/step - loss: 5.8238e-04 - val_loss: 2.1841e-04 - lr: 0.0010
Epoch 6/10
4224/4224 [==============================] - 26s 6ms/step - loss: 5.4792e-04 - val_loss: 1.8251e-04 - lr: 0.0010
Epoch 7/10
4224/4224 [==============================] - 26s 6ms/step - loss: 5.3443e-04 - val_loss: 2.3421e-04 - lr: 0.0010
Epoch 8/10
4224/4224 [==============================] - 26s 6ms/step - loss: 5.1623e-04 - val_loss: 2.7613e-04 - lr: 0.0010
Epoch 9/10
4224/4

/Users/hung/anaconda3/envs/Env3_8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


660/660 [==============================] - 1s 2ms/step
R² Score: 0.993159226961627
Mean Absolute Error (MAE): 0.009545817085317423
Mean Squared Error (MSE): 0.00017296906753442738
Root Mean Squared Error (RMSE): 0.013151770509495191
Training LSTM model with DKCode:  C_CK


...

/Users/hung/anaconda3/envs/Env3_8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


660/660 [==============================] - 1s 2ms/step
R² Score: 0.9809391093270443
Mean Absolute Error (MAE): 0.014117371430700836
Mean Squared Error (MSE): 0.00032003279592468406
Root Mean Squared Error (RMSE): 0.0178894604704749
Training LSTM model with DKCode:  I_I


/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/1540299301.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['lag_24'] = df_corr['lag_24'].fillna(0)
/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/1540299301.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corr['HourUTC'] = pd.to_datetime(df_corr['HourUTC'])
/var/folders/82/s1cgdl3s5hn10rsmff2gt35r0000gn/T/ipykernel_5765/1540299301.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

10
Epoch 1/10
4224/4224 [==============================] - 28s 6ms/step - loss: 0.0030 - val_loss: 3.9488e-04 - lr: 0.0010
Epoch 2/10
4224/4224 [==============================] - 1225s 290ms/step - loss: 7.2349e-04 - val_loss: 2.7990e-04 - lr: 0.0010
Epoch 3/10
4224/4224 [==============================] - 25s 6ms/step - loss: 4.9708e-04 - val_loss: 1.0528e-04 - lr: 0.0010
Epoch 4/10
1775/4224 [===========>..................] - ETA: 14s - loss: 4.4802e-04

KeyboardInterrupt: 